# Modelos de regresión logística

![BLR](https://upload.wikimedia.org/wikipedia/commons/e/ed/Bayes_icon.svg)

En el tema anterior vimos modelos de regresión lineal para modelar variables de comportamiento continuo en relación a otras. En nuestro qué hacer como científicos de datos hay muchos problemas que, por el contrario, se modelan con una variable tipo Bernoulli (solo dos posibilidades, éxito (1) o fracaso (0)). 

En estos contextos, el uso de clasificadores es el apropiado. En este sentido, en esta clase revisaremos cómo implementar un modelo de regresión logística Bayesiana usando PyMC.

> **Objetivos:**
> - Implementar modelos de regresión logística Bayesiana usando PyMC.

> **Referencias:**
> 
> - https://goldinlocks.github.io/Bayesian-logistic-regression-with-pymc3/
> - https://towardsdatascience.com/building-a-bayesian-logistic-regression-with-python-and-pymc3-4dd463bbb16
> - https://www.kaggle.com/datasets/janiobachmann/bank-marketing-dataset

## 1. Descripción del problema

Un banco quiere encontrar cómo mejorar sus estrategias para la siguiente campaña publicitaria. Para esto tenemos datos de campaña(s) publicitaria(s) pasada(s) que se espera que analicemos para encontrar patrones que nos ayuden a encontar ideas para desarrollar futuras estrategias publicitarias.

La campaña publicitaria en cuestión es para que los clientes hagan un depósito a término. Un depósito a término es un depósito en un banco, por el cual el banco ofrece una tasa fija, la cuál será entregada cuando se cumpla el término específico del depósito.

Los atributos de nuestro conjunto de datos son:

1. Datos del cliente:

- age: (numérica)
- job: tipo de trabajo (categórica: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
- marital: estado marital (categórica: 'divorced', 'married', 'single', 'unknown'; note: 'divorced' means divorced or widowed)
- education: (categórica: primary, secondary, tertiary and unknown)
- default: tiene créditos sin pagar? (categórica: 'no', 'yes', 'unknown')
- housing: tiene préstamo hipotecario? (categórica: 'no', 'yes', 'unknown')
- loan: tiene préstamo personal? (categórica: 'no', 'yes', 'unknown')
- balance: balance en su(s) cuenta(s) bancarias.

2. Datos de la campaña publicitaria actual:

- contact: forma de contacto (categórica: 'cellular','telephone')
- month: último mes de contacto en el año (categórica: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
- day: último día de contacto de la semana (categórica: 'mon', 'tue', 'wed', 'thu', 'fri')
- duration: duración del último contacto (numérica). Nota importante: este atributo afecta fuertemente la salida (por ejemplo, si duration=0, entonces y=deposit='no'). Sin embargo, la duración no se sabe antes de que una llamada sea llevada a cabo. También, al final de la llamada y=deposit es obviamente conocida. Por tanto, este atributo debería ser descartado para propósitos de un modelo predictivo.

3. Otros datos:

- campaign: número de contactos llevados a cabo durante la campaña (numérica)
- pdays: número de días que pasaron después de que el cliente fue contactado en una campaña previa (numérica; -1 significa que el cliente no fue contactado previamente)
- previous: número de contactos llevados a cabo antes de esta campaña (numérica)
- poutcome: resultado de la campaña previa (categórica: 'failure', 'unknown', 'success', 'other')


Veamos los datos.

In [ ]:
# Importar pandas


In [ ]:
# Leer el archivo de datos (bank.csv)


In [ ]:
# Cabecera del archivo


In [ ]:
# Descripción de columnas numéricas


In [ ]:
# Descripción de columnas categóricas


## 2. Análisis exploratorio de datos

Antes de proponer algún modelo es importante que nos demos una idea intuitiva de cómo se relacionan los atributos entre sí. Para esto es muy importante realizar un análisis exploratorio de datos.

En este caso haremos un análisis simple, explorando las relaciones entre cada variable y la salida.

In [ ]:
# Importar pyplot

# Importar seaborn


Antes de revisar la relación entre variables, veamos la variable de salida:

In [ ]:
# Histograma de depósitos


Vemos que está prácticamente balanceada.

### Variables numéricas

In [ ]:
# Boxplot de las columnas numéricas, agrupado por la columna "deposit"


Veamos un poco más de cerca el balance:

In [ ]:
# Boxplot de la columna de balance, agrupado por la columna "deposit", acercado


#### Conclusión:

De las variables numéricas, las que parecen tener una relación directa con nuestro objetivo son:

- balance
- duration (esta no la debemos incluir en un modelo predictivo, por las razones antes expuestas)
- pdays
- previous

### Variables categóricas

In [ ]:
# Conteos de las diferentes categorías de las columnas categóricas, agrupado por la columna "deposit"


#### Conclusión:

De las variables categóricas, las que parecen tener una relación directa con nuestro objetivo son:

- default: las personas que han caido en impago tienen menos probabilidad de adquirir un depósito a término. Las personas buena paga están uniformemente distribuidas.
- housing: las personas que tienen un préstamos hipotecario tienen menos probabilidad de adquirir un depósito. Las personas sin préstamo tienen más probabilidad.
- loan: las personas que tienen un préstamo personal tienen menos probabilidad de adquirir un depósito. Las personas sin préstamo están uniformemente distribuidas.
- month: las personas que fueron contactadas en marzo, septiembre, octubre y diciembre tienen más probabilidad de adquirir un depósito. Las personas contactadas en mayo tienen menos probabilidad de adquirir un depósito. En los demás meses la distribución es uniforme.
- poutcome: las personas que aceptaron una campaña publicitaria en el pasado tienen mucha más probabilidad de volverla a aceptar. En los demás casos, la distribución es uniforme.

## 3. Preprocesamiento

Antes de continuar al modelamiento, hagamos las siguientes operaciones:

In [ ]:
# Versión numérica de la columna "deposit"


In [ ]:
# Estandarización de las columnas numéricas


## 4. Regresor logístico Bayesiano

Comencemos con una implementación muy simple, tomando en cuenta solamente las variables numéricas.

Los regresores logísticos se parecen bastante a los regresores lineales. En este caso, al ser un problema de clasificación binaria, nuestra salida ("deposit") la modelamos con una distribución Bernoulli.

$$
D_i \sim \text{Bernoulli}(p_i).
$$

La probabilidad, es la que dependerá de los predictores ("balance", "pdays" y "previous"). Sabemos que la probabilidad debe de estar entre 0 y 1, por lo que usamos la función logit para asegurar esto:

$$
\text{logit}(p_i) = \log \frac{p_i}{1 - p_i}.
$$

Es sobre $\text{logit}(p_i)$ que definimos un modelo lineal de los predictores:

$$
\text{logit}(p_i) = \alpha + \beta_{balance}balance_i + \beta_{pdays}pdays_i + \beta_{previous}previous_i.
$$

Para los parámetros $\alpha$, $\beta_{balance}$, $\beta_{pdays}$, y $\beta_{previous}$ definimos previas. El modelo queda como sigue:

$$
\begin{align}
\begin{array}{lcl}
deposit_i & \sim & \text{Bernoulli}(p_i) \\
\text{logit}(p_i) & = & \alpha + \beta_{balance}balance_i + \beta_{pdays}pdays_i + \beta_{previous}previous_i \\
\alpha & \sim & \text{Normal}(0, 1) \\
\beta_{balance} & \sim & \text{Normal}(0, 1) \\
\beta_{pdays} & \sim & \text{Normal}(0, 1) \\
\beta_{previous} & \sim & \text{Normal}(0, 1) \\
\end{array}
\end{align}
$$

In [ ]:
# Importar pymc

# Importar arviz

# Importar numpy


In [ ]:
# Modelo solo con variables numéricas
with pm.Model() as only_numeric:
    # Previas
    
    # Regresión
    
    # Deposit
    


### Simulación predictiva previa

Veamos que tan buenas son estas previas que elegimos:

In [ ]:
# Simulación predictiva previa


In [ ]:
# Objeto prior predictive


In [ ]:
# Generemos algunas de las densidades previas para algunos sujetos


Las previas elegidas parecen hacer un buen trabajo, en el sentido de que las probabilidades no se cargan a 0 o a 1.

### Resultados e interpretación

Generemos muestras de la distribución posterior:

In [ ]:
# Muestreo de la distribución posterior


Veamos las posteriores para las $\beta$ s:

In [ ]:
# az.plot_posterior


Observamos que en conjunto las tres variables, en efecto tienen una relación positiva en la probabilidad de que un sujeto adquiera un depósito.

Es importante recordar que el espacio en el que están estas $\beta$ s, es un espacio transformado por la transformación logística inversa.

Para $\alpha$, debido a que estandarizamos nuestras variables, sabemos que $\text{logit}(p) = \alpha$ es la probabilidad de que un sujeto con balance, pdays y previous promedios acepte un depósito.

In [ ]:
# Importar scipy.special.expit as logistic


In [ ]:
# az.plot_posterior


La probabilidad de que un sujeto promedio acepte un depósito está entre 0.47 y 0.49.

### Evaluación del modelo

Comenzamos por hacer una partición de los datos:

In [ ]:
# Importar train_test_split


In [ ]:
# Partición de los datos en entrenamiento y prueba


Redefinimos el modelo usando variables mutables, con los datos de entrenamiento:

In [ ]:
# Redefinimos modelo con variables mutables
with pm.Model() as only_numeric_mutable:
    # Variables numéricas mutables
    
    # Previas
    
    # Regresión
    
    # Deposit
    

Muestreamos la distribución posterior:

In [ ]:
# Muestreo de la distribución posterior


Para los datos observados, muestreamos la distribución posterior predictiva:

In [ ]:
# Cambiamos los datos observados y muestreamos la distribución posterior predictiva


Observamos la posterior predictiva:

In [ ]:
# Tenemos 4 cadenas, 1000 muestras por cadena y 2233 sujetos de prueba


In [ ]:
# Tomamos promedio sobre las cadenas y las muestras


Con lo anterior, llevamos a cabo la evaluación, eligiendo el mejor threshold para maximizar el f1:

In [ ]:
# Importamos precision_recall_curve y f1_score


In [ ]:
# Construimos la curva precision-recall


In [ ]:
# Construimos la curva F1 vs. umbrales

# Encontramos el umbral que maximiza F1


In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(13, 5))

# Curva Precision-Recall, con el mejor umbral marcado
sns.scatterplot(x='Recall', y='Precision', data=pr_curve, ax=axes[0])
axes[0].plot(
    pr_curve.loc[np.where(thresholds == best_threshold)[0], 'Recall'],
    pr_curve.loc[np.where(thresholds == best_threshold)[0], 'Precision'],
    'ro',
    ms=10
)
axes[0].set_ylim(0,1)
axes[0].set_title('Precision-Recall Curve')

# Curva F1 vs. umbrales, con el mejor umbral marcado
f1.plot(ax=axes[1], title='F1 Scores', ylim=(0,1))
axes[1].set_xlabel('Threshold')
axes[1].axvline(best_threshold, lw=1, ls='--', color='k')
axes[1].text(x=.60, y=.95, s=f'Max F1 @ {best_threshold:.2f}')
fig.tight_layout()
plt.subplots_adjust(top=.8)
plt.show();

In [ ]:
# Accuracy


<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>